# Gemini API: JSON Mode Quickstart

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/quickstarts/JSON_mode.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>

The Gemini API can be used to generate a JSON output if we set the schema that we would like to use.

**Note**: Use Gemini 1.5 Pro when generating JSON. JSON schemas are only supported by Gemini 1.5 Pro right now.

In [1]:
!pip install -U -q google-generativeai

In [2]:
import google.generativeai as genai
from google.generativeai.types import content_types

import json
import dataclasses
import typing_extensions as typing

## Configure your API key

To run the following cell, your API key must be stored in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see [Authentication](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Authentication.ipynb) for an example.

In [3]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

## Activate JSON Mode

 Activate JSON mode by specifying `respose_mime_type` in the `generation_config` parameter. We'll use Gemini 1.5 Pro for this example, but note that you can use Gemini 1.5 Flash while describing your schema in the prompt.

In [18]:
model = genai.GenerativeModel("gemini-1.5-pro-latest",
                              generation_config={"response_mime_type": "application/json"})

In [12]:
class Recipe(typing.TypedDict):
    recipe_name: str

In [13]:
response_schema = content_types._schema_for_class(list[Recipe])
response_schema

{'items': {'properties': {'recipe_name': {'type': 'string'}},
  'type': 'object'},
 'type': 'array'}

In [29]:
prompt = """List a few popular cookie recipes using this JSON schema, be sure to return an array:
{'type': 'object', 'properties': { 'recipe_name': {'type': 'string'}}}"""

In [30]:
response = model.generate_content(prompt)
print(response.text)

[{"recipe_name": "Chocolate Chip Cookies"}, {"recipe_name": "Peanut Butter Cookies"}, {"recipe_name": "Oatmeal Raisin Cookies"}, {"recipe_name": "Sugar Cookies"}, {"recipe_name": "Snickerdoodles"}]



Just for fun, parse the string to JSON, and then serialize it.

In [31]:
print(json.dumps(json.loads(response.text), indent=4))

[
    {
        "recipe_name": "Chocolate Chip Cookies"
    },
    {
        "recipe_name": "Peanut Butter Cookies"
    },
    {
        "recipe_name": "Oatmeal Raisin Cookies"
    },
    {
        "recipe_name": "Sugar Cookies"
    },
    {
        "recipe_name": "Snickerdoodles"
    }
]


## Generate JSON from schema

We can take a Python class, for instance, and use it as our schema for generating JSON. When passing in the `response_schema` parameter, use the Gemini 1.5 Pro model. Gemini 1.5 Flash does not support this.

In [5]:
class Person(typing.TypedDict):
    family_name: str
    favorite_food: str

In [6]:
prompt = "Hello, describe some people, all fields are required."

In [8]:
response_schema = content_types._schema_for_class(list[Person])
response_schema

{'items': {'properties': {'family_name': {'type': 'string'},
   'favorite_food': {'type': 'string'}},
  'type': 'object'},
 'type': 'array'}

Using `generate_content`, we pass in the Python class `Person` defined above into the `generation_config`'s `response_schema` field.

In [9]:
model = genai.GenerativeModel(model_name="models/gemini-1.5-pro-latest")

result = model.generate_content(
    prompt,
    generation_config={"response_mime_type": "application/json",
                       "response_schema": response_schema,
    },
    request_options={"timeout": 600},
)

In [10]:
print(result.text)

[{"favorite_food": "pizza"}] 
